In [2]:
! pip install cssselect

In [3]:
import requests
import numpy as np
import pandas as pd
import json
import joblib
import cssselect
import lxml.html
from tqdm import tqdm

In [7]:
train = pd.read_csv('/Users/baeyuna/Documents/SNU_Dlab/Data/MINDlarge_train/news.tsv', delimiter='\t', header=None)
print(train.shape)
train.head()

(101527, 8)


,0,1,2,3,4,5,6,7
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."


In [10]:
train.iloc[:5,5].tolist()

['https://assets.msn.com/labs/mind/AAGH0ET.html',
 'https://assets.msn.com/labs/mind/AABmf2I.html',
 'https://assets.msn.com/labs/mind/AAB19MK.html',
 'https://assets.msn.com/labs/mind/AAISxPN.html',
 'https://assets.msn.com/labs/mind/AAJgNsz.html']

In [ ]:
train_urls = train.loc[:,5]

In [11]:
dev = pd.read_csv('/Users/baeyuna/Documents/SNU_Dlab/Data/MINDlarge_dev/news.tsv', delimiter='\t', header=None)
print(dev.shape)
dev.head()

(72023, 8)


,0,1,2,3,4,5,6,7
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
2,N86255,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
3,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
4,N75236,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://assets.msn.com/labs/mind/AACk2N6.html,[],"[{""Label"": ""National Basketball Association"", ..."


In [ ]:
def getReviews(idx):
    reviews_list = []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'
    }
    
    for page in [0,1,2]:
        try: # 페이지가 있으면
            url = f'https://store.naver.com/restaurants/detail?entry=plt&id={idx}&tab=receiptReview&tabPage={page}'
            response = requests.get(url, headers = headers)
            root = lxml.html.fromstring(response.text)
            
            nicks = [elem.text_content().strip() for elem in root.cssselect('div.reviewer > a')]
            reviews = [elem.text_content().strip() for elem in root.cssselect('div.review_txt')]
            stars = [elem.attrib['style'] for elem in root.cssselect('span.bg > span.value')]
            herfs = [elem.attrib['href'].split('my/')[1].split('/')[0] for elem in root.cssselect('a.item')]
            
            value = list(map(list, zip(nicks, reviews, stars, herfs)))
            [x.append(idx) for x in value]
            reviews_list.extend(value)
        except: # 페이지가 없으면
            continue

    return reviews_list

In [12]:
import os

In [16]:
train.head()

,0,1,2,3,4,5,6,7
0,N88753,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://assets.msn.com/labs/mind/AAGH0ET.html,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N45436,news,newsscienceandtechnology,Walmart Slashes Prices on Last-Generation iPads,Apple's new iPad releases bring big deals on l...,https://assets.msn.com/labs/mind/AABmf2I.html,"[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ...","[{""Label"": ""IPad"", ""Type"": ""J"", ""WikidataId"": ..."
2,N23144,health,weightloss,50 Worst Habits For Belly Fat,These seemingly harmless habits are holding yo...,https://assets.msn.com/labs/mind/AAB19MK.html,"[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Adipose tissue"", ""Type"": ""C"", ""Wik..."
3,N86255,health,medical,Dispose of unwanted prescription drugs during ...,NaN,https://assets.msn.com/labs/mind/AAISxPN.html,"[{""Label"": ""Drug Enforcement Administration"", ...",[]
4,N93187,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://assets.msn.com/labs/mind/AAJgNsz.html,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."


In [15]:
start_urls = []
with open('/Users/baeyuna/Documents/SNU_Dlab/Data/MINDlarge_train/news.tsv', 'r') as f:
    for l in f:
        _, _, _, _, _, url, _, _ = l.strip('\n').split('\t')
        start_urls.append(url)

ValueError: not enough values to unpack (expected 9, got 8)